# LLM Email Generator 

### Create an LLM object and test it

In [16]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model_name="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key='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'
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


### Web scraper to get data from website

In [17]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-42684")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Principal Software Engineer

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologyPrincipal Software EngineerSan Francisco, CaliforniaBecome a Part of the NIKE, Inc. TeamN

### Prompt LLM to create a JSON summary of the information

In [18]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ###  INSTRUCTION:
    The scraped text is from a career page of a website.
    Your job is to extract the job postings and return them in JSON format containing
    the following keys: `role`, `experience`, `skills` and `description`.
    Only return the valid JSON
    ### VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data': page_data})
print(res.content)

```json
{
  "role": "Principal Software Engineer",
  "experience": "2 years of experience in the job offered or in a computer-related occupation",
  "skills": [
    "AWS",
    "Java",
    "Javascript",
    "SEO",
    "SQL",
    "CI/CD",
    "Azure",
    "CDN and WAF",
    "Splunk"
  ],
  "description": "Create and manage roadmaps with product and engineering leadership; Provide guidance on approaches to the implementation and realization of roadmaps; Collaborate with multiple multi-functional engineering teams within the company to build future product; Drive cross-domain alignment and technical delivery of major capabilities across value streams; Assess solution options based on their relative merits and risks to determine the best solution; Provide technical leadership in the evaluation, conceptualization, design, and development of major new capabilities; Assist in the establishment of core platforms and build a capability maturity vision to progress and improve our platforms; Archi

In [19]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Principal Software Engineer',
 'experience': '2 years of experience in the job offered or in a computer-related occupation',
 'skills': ['AWS',
  'Java',
  'Javascript',
  'SEO',
  'SQL',
  'CI/CD',
  'Azure',
  'CDN and WAF',
  'Splunk'],
 'description': 'Create and manage roadmaps with product and engineering leadership; Provide guidance on approaches to the implementation and realization of roadmaps; Collaborate with multiple multi-functional engineering teams within the company to build future product; Drive cross-domain alignment and technical delivery of major capabilities across value streams; Assess solution options based on their relative merits and risks to determine the best solution; Provide technical leadership in the evaluation, conceptualization, design, and development of major new capabilities; Assist in the establishment of core platforms and build a capability maturity vision to progress and improve our platforms; Architect and develop sophisticated solutio

In [20]:
type(json_res)

dict

### Store portfolio projects with corresponding links in vector database

In [22]:
import pandas as pd

df = pd.read_csv('my_portfolio.csv')
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [24]:
import uuid
import chromadb

# normal client creates db in memory
# persistentclient creates db on disk, and data can be reused later
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfolio')

if not collection.count():
    for  _, row in df.iterrows():
        collection.add(documents=row['Techstack'],
                       metadatas={'links': row['Links']},
                       ids=[str(uuid.uuid4())])

In [25]:
links = collection.query(query_texts=["Experience in Python", "Experience in React"], n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [26]:
job = json_res
job['skills']

['AWS',
 'Java',
 'Javascript',
 'SEO',
 'SQL',
 'CI/CD',
 'Azure',
 'CDN and WAF',
 'Splunk']

### Prompt template to create the email

In [29]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    
    ### INSTRUCTION:
    You are Bob, a business development executive at ClosedAI. ClosedAI is an AI & Software Consulting company dedicated to facilitating
    the seamless integration of business processes through automated tools. 
    Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
    process optimization, cost reduction, and heightened overall efficiency. 
    Your job is to write a cold email to the client regarding the job mentioned above describing the capability of ClosedAI 
    in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase ClosedAI's portfolio: {link_list}
    Remember you are Bob, BDE at ClosedAI. 
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
    
    """
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Solutions for Your Business Needs

Dear Hiring Manager,

I came across the job description for a Principal Software Engineer at your esteemed organization and was impressed by the scope of responsibilities and the cutting-edge technologies involved. As a Business Development Executive at ClosedAI, I'd like to introduce our company as a trusted partner in delivering top-notch software engineering solutions that align with your business objectives.

At ClosedAI, we specialize in providing AI-powered and software consulting services that cater to the unique needs of our clients. Our team of expert engineers and developers has a proven track record of designing, developing, and deploying sophisticated solutions that drive business growth, scalability, and efficiency.

Our expertise in AWS, Java, JavaScript, SQL, CI/CD, Azure, CDN and WAF, and Splunk aligns perfectly with the requirements of the Principal Software Engineer role. We've successfully implem